In [ ]:
# Start with the blank canvas
graph_builder = StateGraph(State)

graph_builder = graph_builder.add_sequence([retrieve, generate])

graph_builder.add_edge(START, "retrieve")



In [ ]:
from langgraph.graph import StateGraph, START
from typing import TypedDict, List, Optional

# -------------------------
# Define State structure
# -------------------------

class State(TypedDict):
    question: str
    documents: Optional[List[str]]
    response: Optional[str]
    verdict: Optional[str]
    attempts: int

# -------------------------
# Define nodes
# -------------------------

def retrieve(state):
    # Dummy simulated retrieval
    state["documents"] = ["PDF snippet 1", "PDF snippet 2"]
    return state

def check_context(state):
    if not state.get("documents"):
        state["response"] = "I don't know (no context found)"
        state["stop"] = True
    return state

def generate(state):
    context = " ".join(state["documents"])
    question = state["question"]
    state["response"] = f"Answering: {question}\nUsing context: {context}"
    return state

def fact_check(state):
    # Dummy simulated verdict
    if "edge case" in state["question"].lower():
        state["verdict"] = "No"
    else:
        state["verdict"] = "Yes"
    return state

def end(state):
    return state

# -------------------------
# Build graph
# -------------------------

graph = StateGraph(State)

graph = graph.add_node("retrieve", retrieve)
graph = graph.add_node("check_context", check_context)
graph = graph.add_node("generate", generate)
graph = graph.add_node("fact_check", fact_check)
graph = graph.add_node("end", end)

# Add main sequence
graph = graph.add_edge(START, "retrieve")
graph = graph.add_edge("retrieve", "check_context")

# If context exists, go to generate
graph = graph.add_edge("check_context", "generate")
# If no context, skip to end (check_context sets stop flag)
graph = graph.add_edge("check_context", "end")

graph = graph.add_edge("generate", "fact_check")

# Conditional edges for fact_check
def check_verdict(state):
    attempts = state.get("attempts", 0)
    if "yes" in state.get("verdict", "").lower():
        return "pass"
    else:
        attempts += 1
        state["attempts"] = attempts
        if attempts >= 3:
            state["response"] = "I don't know (max attempts reached)"
            return "pass"
        return "fail"

graph = graph.add_conditional_edges(
    "fact_check",
    check_verdict,
    {
        "pass": "end",
        "fail": "generate"
    }
)

# -------------------------
# Compile and run
# -------------------------

compiled_graph = graph.compile()

state = {
    "question": "How could we extend our current implementation to handle edge cases?",
    "attempts": 0,
}

result = compiled_graph.invoke(state)
print(result["response"])
